### Install lib 

In [182]:
%pip install meteostat pandas xlrd datetime

Note: you may need to restart the kernel to use updated packages.


#### Data meteostat: Historical weather data of 8 departments in ile de france 


In [183]:
from meteostat import Point, Daily,Hourly
from datetime import datetime
import pandas as pd

In [184]:
# Define 8 coordinates
idf_points = {
    "Paris": (48.8566, 2.3522),
    "Hauts-de-Seine": (48.8924, 2.2060), # Nanterre
    "Seine-Saint-Denis": (48.9109, 2.4395), # Bobigny
    "Val-de-Marne": (48.7904, 2.4556), #Créteil
    "Yvelines": (48.8049, 2.1204), # Versailles
    "Essonne": (48.6290, 2.4410), # Evry
    "Val-d'Oise ": (49.0360, 2.0630), # Cergy
    "Seine-et-Marne": (48.5390, 2.6550) #Melun
}

# Define time range
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)

# Retrieve daily data for each city
dfs = []
for city, (lat, lon) in idf_points.items():
    p = Point(lat, lon)
    data = Hourly(p, start, end).fetch().reset_index()
    data['city'] = city
    dfs.append(data)

weather_all = pd.concat(dfs, ignore_index=True)
weather_all



,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,city
0,2024-01-01 00:00:00,7.1,3.9,80.0,0.0,0.0,230.0,16.6,46.0,1007.2,0.0,2.0,Paris
1,2024-01-01 01:00:00,7.4,4.2,80.0,0.0,<NA>,240.0,16.6,42.6,1007.3,0.0,2.0,Paris
2,2024-01-01 02:00:00,7.3,3.9,79.0,0.0,<NA>,230.0,18.4,42.6,1007.8,0.0,2.0,Paris
3,2024-01-01 03:00:00,7.0,3.6,79.0,0.0,0.0,230.0,20.5,31.0,1008.0,0.0,2.0,Paris
4,2024-01-01 04:00:00,7.0,3.2,77.0,0.0,<NA>,230.0,18.4,37.0,1008.5,0.0,3.0,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70083,2024-12-30 20:00:00,1.0,0.6,97.0,0.0,<NA>,210.0,11.2,16.7,1032.2,0.0,3.0,Seine-et-Marne
70084,2024-12-30 21:00:00,0.8,0.4,97.0,0.0,0.0,230.0,13.0,22.0,1032.0,0.0,3.0,Seine-et-Marne
70085,2024-12-30 22:00:00,0.8,0.5,98.0,0.0,<NA>,240.0,13.0,20.4,1032.1,0.0,3.0,Seine-et-Marne
70086,2024-12-30 23:00:00,0.8,0.5,98.0,0.0,<NA>,210.0,9.4,20.4,1032.4,0.0,5.0,Seine-et-Marne


In [185]:
weather_all.isnull().sum()

time        0
temp        0
dwpt        0
rhum        0
prcp      917
snow    51083
wdir        0
wspd        0
wpgt     8761
pres        0
tsun     8761
coco      251
city        0
dtype: int64

In [186]:
weather_all.describe()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,70088,70088.0,70088.0,70088.0,69171.0,19005.0,70088.0,70088.0,61327.0,70088.0,61327.0,69837.0
mean,2024-07-01 12:00:00,12.619331,8.716628,79.160727,0.107233,0.01768,197.08408,12.467308,23.498744,1016.214854,7.957653,3.715595
min,2024-01-01 00:00:00,-6.9,-10.7,25.0,0.0,0.0,0.0,0.0,0.0,981.0,0.0,1.0
25%,2024-04-01 06:00:00,8.3,5.4,69.0,0.0,0.0,130.0,7.6,14.8,1011.0,0.0,2.0
50%,2024-07-01 12:00:00,12.3,9.1,83.0,0.0,0.0,210.0,11.2,21.0,1016.3,0.0,3.0
75%,2024-09-30 18:00:00,16.9,12.4,92.0,0.0,0.0,270.0,16.6,29.6,1023.0,12.0,4.0
max,2024-12-31 00:00:00,37.0,23.9,100.0,20.0,7.0,360.0,63.0,91.0,1038.6,60.0,25.0
std,NaN,6.558909,5.172106,15.378182,0.50401,0.26115,96.76033,6.779357,11.591247,10.094825,12.984435,2.894431


#### Drop snow, tsun


In [187]:
feature_meteo = ["time","temp","dwpt","rhum","prcp","wdir","wspd","pres","coco","city"]


In [188]:
df_weather = weather_all[feature_meteo]

#### Data electricity ile de france

In [189]:
ele = pd.read_csv( "ile_de_france_ele.xls",sep= "\t",encoding="latin-1")  
ele.head()

,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Unnamed: 29
0,Ile-de-France,Données consolidées,2024-01-01,00:00,7843.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.61,86.9,0.0,0.0,0.01,5.26,1.81,43.96,NaN
1,Ile-de-France,Données consolidées,2024-01-01,00:15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ile-de-France,Données consolidées,2024-01-01,00:30,8043.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.57,86.9,0.0,0.0,0.01,5.26,1.82,45.20,NaN
3,Ile-de-France,Données consolidées,2024-01-01,00:45,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ile-de-France,Données consolidées,2024-01-01,01:00,7731.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.63,86.9,0.0,0.0,0.01,5.26,1.86,44.58,NaN


In [190]:
ele.columns

Index(['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Ech. physiques',
       'TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)',
       'TCH Nucléaire (%)', 'TCO Eolien (%)', 'TCH Eolien (%)',
       'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
       'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)',
       'Unnamed: 29'],
      dtype='object')

In [191]:
# drop unamed: 29
feature_ele = ['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Ech. physiques']
df_ele = ele[feature_ele]

# TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)'
#        , 'TCO Eolien (%)', 'TCH Eolien (%)',
#        'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
#        'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)'

In [192]:
df = df_ele.dropna()

In [193]:
df_ele.shape

(35137, 17)

In [194]:
ele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35137 entries, 0 to 35136
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Périmètre            35137 non-null  object 
 1   Nature               35136 non-null  object 
 2   Date                 35136 non-null  object 
 3   Heures               35136 non-null  object 
 4   Consommation         17568 non-null  float64
 5   Thermique            17568 non-null  float64
 6   Nucléaire            17568 non-null  float64
 7   Eolien               17568 non-null  float64
 8   Solaire              17568 non-null  float64
 9   Hydraulique          17568 non-null  float64
 10  Pompage              17568 non-null  float64
 11  Bioénergies          17568 non-null  float64
 12  Stockage batterie    17568 non-null  float64
 13  Déstockage batterie  17568 non-null  float64
 14  Eolien terrestre     17568 non-null  float64
 15  Eolien offshore      17568 non-null 

In [195]:
# Rename colunms ele
ele = df.rename(columns = {"Périmètre":"Perimeter",
                            "Nucléaire":"Nuclear",
                            "Bioénergies":"Bioenergies",
                            "Déstockage batterie ":"Battery clearance",
                            "TCO Nucléaire (%)":"TCO Nuclear (%)",
                            "TCH Nucléaire (%)":"TCH Nuclear",
                            "TCO Bioénergies (%)":"TCO Bioenergies (%)",
                            "TCH Bioénergies(%)":"TCh Bioenergies (%)"})

In [196]:
ele

,Perimeter,Nature,Date,Heures,Consommation,Thermique,Nuclear,Eolien,Solaire,Hydraulique,Pompage,Bioenergies,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,Ech. physiques
0,Ile-de-France,Données consolidées,2024-01-01,00:00,7843.0,197.0,0.0,126.0,0.0,1.0,0.0,142.0,0.0,0.0,0.0,0.0,7379.0
2,Ile-de-France,Données consolidées,2024-01-01,00:30,8043.0,197.0,0.0,126.0,0.0,1.0,0.0,146.0,0.0,0.0,0.0,0.0,7573.0
4,Ile-de-France,Données consolidées,2024-01-01,01:00,7731.0,197.0,0.0,126.0,0.0,1.0,0.0,144.0,0.0,0.0,0.0,0.0,7264.0
6,Ile-de-France,Données consolidées,2024-01-01,01:30,7559.0,198.0,0.0,127.0,0.0,1.0,0.0,140.0,0.0,0.0,0.0,0.0,7093.0
8,Ile-de-France,Données consolidées,2024-01-01,02:00,7415.0,199.0,0.0,124.0,0.0,1.0,0.0,146.0,0.0,0.0,0.0,0.0,6946.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35126,Ile-de-France,Données consolidées,2024-12-31,21:30,9699.0,149.0,0.0,44.0,0.0,2.0,0.0,140.0,0.0,0.0,0.0,0.0,9363.0
35128,Ile-de-France,Données consolidées,2024-12-31,22:00,9480.0,150.0,0.0,54.0,0.0,2.0,0.0,139.0,0.0,0.0,0.0,0.0,9136.0
35130,Ile-de-France,Données consolidées,2024-12-31,22:30,9599.0,150.0,0.0,67.0,0.0,2.0,0.0,141.0,0.0,0.0,0.0,0.0,9237.0
35132,Ile-de-France,Données consolidées,2024-12-31,23:00,9727.0,148.0,0.0,68.0,0.0,2.0,0.0,142.0,0.0,0.0,0.0,0.0,9364.0


In [197]:
ele.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17568 entries, 0 to 35134
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Perimeter            17568 non-null  object 
 1   Nature               17568 non-null  object 
 2   Date                 17568 non-null  object 
 3   Heures               17568 non-null  object 
 4   Consommation         17568 non-null  float64
 5   Thermique            17568 non-null  float64
 6   Nuclear              17568 non-null  float64
 7   Eolien               17568 non-null  float64
 8   Solaire              17568 non-null  float64
 9   Hydraulique          17568 non-null  float64
 10  Pompage              17568 non-null  float64
 11  Bioenergies          17568 non-null  float64
 12  Stockage batterie    17568 non-null  float64
 13  Déstockage batterie  17568 non-null  float64
 14  Eolien terrestre     17568 non-null  float64
 15  Eolien offshore      17568 non-null  floa

In [198]:
ele['Datetime'] = pd.to_datetime(ele['Date'].astype(str) + ' ' + ele['Heures'].astype(str), errors='coerce')
ele = ele.dropna(subset=['Datetime']).sort_values('Datetime')
ele = ele.set_index('Datetime')
ele_hourly = (ele.resample('1h').agg({
                                    'Consommation': 'mean',
                                    'Thermique': 'mean',
                                    'Nuclear': 'mean',
                                    'Eolien': 'mean',
                                    'Solaire': 'mean',
                                    'Hydraulique': 'mean',
                                    'Pompage': 'mean',
                                    'Bioenergies': 'mean',
                                    'Stockage batterie': 'mean',
                                    'Déstockage batterie': 'mean',
                                    'Eolien terrestre': 'mean',
                                    'Eolien offshore': 'mean',
                                    'Ech. physiques': 'mean'
                                    })
                                    .reset_index()
)

ele_hourly.rename(columns={'Datetime': 'time'}, inplace=True)
ele_hourly

,time,Consommation,Thermique,Nuclear,Eolien,Solaire,Hydraulique,Pompage,Bioenergies,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,Ech. physiques
0,2024-01-01 00:00:00,7943.0,197.0,0.0,126.0,0.0,1.0,0.0,144.0,0.0,0.0,0.0,0.0,7476.0
1,2024-01-01 01:00:00,7645.0,197.5,0.0,126.5,0.0,1.0,0.0,142.0,0.0,0.0,0.0,0.0,7178.5
2,2024-01-01 02:00:00,7338.5,199.0,0.0,121.5,0.0,1.0,0.0,144.0,0.0,0.0,0.0,0.0,6873.5
3,2024-01-01 03:00:00,7159.0,199.0,0.0,105.0,0.0,1.0,0.0,141.5,0.0,0.0,0.0,0.0,6713.5
4,2024-01-01 04:00:00,7016.0,199.0,0.0,85.5,0.0,1.0,0.0,137.5,0.0,0.0,0.0,0.0,6595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2024-12-31 19:00:00,11185.5,149.0,0.0,45.5,0.0,2.0,0.0,143.5,0.0,0.0,0.0,0.0,10846.5
8780,2024-12-31 20:00:00,10621.0,148.5,0.0,40.5,0.0,2.0,0.0,143.5,0.0,0.0,0.0,0.0,10291.0
8781,2024-12-31 21:00:00,9886.5,149.0,0.0,42.0,0.0,2.0,0.0,140.5,0.0,0.0,0.0,0.0,9558.0
8782,2024-12-31 22:00:00,9539.5,150.0,0.0,60.5,0.0,2.0,0.0,140.0,0.0,0.0,0.0,0.0,9186.5


#### source population : https://idf.handidonnees.fr/page/population

In [199]:
population_idf = {
    "Paris": 2048472,
    "Hauts-de-Seine": 1663974,
    "Seine-Saint-Denis": 1711876,
    "Val-de-Marne": 1426433,
    "Yvelines": 1489363,
    "Essonne": 1343527,
    "Val-d'Oise": 1287589,
    "Seine-et-Marne": 1479615
}
for ele,items in population_idf.items():
    print(ele,items)

Paris 2048472
Hauts-de-Seine 1663974
Seine-Saint-Denis 1711876
Val-de-Marne 1426433
Yvelines 1489363
Essonne 1343527
Val-d'Oise 1287589
Seine-et-Marne 1479615


#### In the reason of disable of data electricity in each department of ile de france, so we need to apply a method that we can obtain from an approximate of utilization of each department base on the each number population of the level aggregation in the IDF. Hence we have the method as follow: 
#### Let: <br>
&nbsp; $E_{d,t}$ &nbsp; is an utilization of the electricity in department d at the ti me t. <br>
&nbsp;   $E_{IDF,t}$ &nbsp; is an utilization of the electricity in IDF at the time t.  <br>
&nbsp;  $Pop_d$ &nbsp; is number of population in departement d. <br>

$$E_{d,t} = E_{IDF,t} \times \frac{Pop_d}{\sum_d{Pop_d}}$$

In [200]:
total_pop = sum(population_idf.values())

In [ ]:
# create new columns cosommation electricity base on population in each department 
ele_dept = (
    pd.DataFrame({
        'time': pd.Series(ele_hourly['time'].values.repeat(len(population_idf))),
        'Department': list(population_idf.keys()) * len(ele_hourly),
    })
    .merge(ele_hourly[['time','Consommation']], on='time', how='left')
)

ele_dept['pop_weight'] = ele_dept['Department'].map(population_idf) / total_pop
ele_dept['Electricity_MW'] = ele_dept['Consommation'] * ele_dept['pop_weight']
ele_dept = ele_dept.drop(columns=['pop_weight'])
ele_dept


,time,Department,Consommation,Electricity_MW
0,2024-01-01 00:00:00,Paris,7943.0,1306.819567
1,2024-01-01 00:00:00,Hauts-de-Seine,7943.0,1061.529658
2,2024-01-01 00:00:00,Seine-Saint-Denis,7943.0,1092.088665
3,2024-01-01 00:00:00,Val-de-Marne,7943.0,909.990742
4,2024-01-01 00:00:00,Yvelines,7943.0,950.136839
...,...,...,...,...
70267,2024-12-31 23:00:00,Val-de-Marne,9665.5,1107.329160
70268,2024-12-31 23:00:00,Yvelines,9665.5,1156.181243
70269,2024-12-31 23:00:00,Essonne,9665.5,1042.969858
70270,2024-12-31 23:00:00,Val-d'Oise,9665.5,999.545612


In [206]:
weather_name_map = {
    'Hauts-de-Seine':'Hauts-de-Seine',
    'Seine-Saint-Denis':'Seine-Saint-Denis',
    'Val-de-Marne':'Val-de-Marne',
    'Yvelines':'Yvelines',
    'Essonne':'Essonne',
    "Val-d'Oise":"Val-d'Oise",
    'Seine-et-Marne':'Seine-et-Marne'
}
weather_long = weather_all.copy()
weather_long['Department'] = weather_long['city'].map(weather_name_map)
weather_long = weather_long.dropna(subset=['Department'])

df_dep = (
    weather_long
    .merge(ele_dept[['time','Department','Electricity_MW']], on=['time','Department'], how='inner')
    .sort_values(['Department','time']).reset_index()
)
df_dep = df_dep.drop(columns=['index','snow','city'])
df_dep


,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,tsun,coco,Department,Electricity_MW
0,2024-01-01 00:00:00,7.0,3.8,80.0,0.0,230.0,22.3,46.0,1007.6,0.0,2.0,Essonne,857.100987
1,2024-01-01 01:00:00,6.9,4.0,82.0,0.0,240.0,22.3,42.6,1007.8,0.0,2.0,Essonne,824.944862
2,2024-01-01 02:00:00,6.3,3.6,83.0,0.0,230.0,16.6,42.6,1008.3,0.0,2.0,Essonne,791.871533
3,2024-01-01 03:00:00,6.1,3.4,83.0,0.0,230.0,18.4,31.0,1008.7,0.0,2.0,Essonne,772.502324
4,2024-01-01 04:00:00,5.9,3.1,82.0,0.0,240.0,18.4,37.0,1009.2,0.0,2.0,Essonne,757.071701
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52561,2024-12-30 20:00:00,0.3,0.2,99.0,0.0,200.0,9.4,16.7,1031.9,0.0,5.0,Yvelines,1328.612598
52562,2024-12-30 21:00:00,-0.2,-0.2,100.0,0.0,200.0,13.0,22.0,1032.0,0.0,5.0,Yvelines,1255.644769
52563,2024-12-30 22:00:00,0.1,0.1,100.0,0.0,210.0,9.4,20.4,1032.0,0.0,5.0,Yvelines,1222.869055
52564,2024-12-30 23:00:00,0.3,0.3,100.0,0.0,220.0,9.4,20.4,1032.3,0.0,5.0,Yvelines,1220.835525


In [208]:
df_dep.isnull().sum()

time                0
temp                0
dwpt                0
rhum                0
prcp              734
wdir                0
wspd                0
wpgt                0
pres                0
tsun                0
coco                0
Department          0
Electricity_MW      0
dtype: int64

In [209]:
df_dep.describe()

,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,tsun,coco,Electricity_MW
count,52566,52566.0,52566.0,52566.0,51832.0,52566.0,52566.0,52566.0,52566.0,52566.0,52566.0,52566.000000
mean,2024-07-01 12:00:00,12.652249,8.771527,79.358768,0.107258,196.83476,12.458806,23.516421,1016.244323,7.955808,3.773827,896.993527
min,2024-01-01 00:00:00,-6.4,-10.3,25.0,0.0,0.0,0.0,0.0,981.0,0.0,1.0,414.576607
25%,2024-04-01 06:00:00,8.2,5.5,69.0,0.0,130.0,7.6,14.8,1011.0,0.0,2.0,725.268305
50%,2024-07-01 12:00:00,12.3,9.1,83.0,0.0,210.0,11.2,22.0,1016.3,0.0,3.0,862.927935
75%,2024-09-30 18:00:00,17.0,12.4,92.0,0.0,270.0,16.6,29.6,1023.0,12.0,4.0,1042.541497
max,2024-12-31 00:00:00,37.0,23.9,100.0,20.0,360.0,63.0,91.0,1038.6,60.0,25.0,1915.795476
std,NaN,6.594598,5.17135,15.656333,0.510364,97.067493,6.892241,11.58671,10.075833,12.987473,2.914482,242.396121
